In [20]:
# Cell 1: Imports

import kuzu
import os
import shutil
import pandas as pd

In [21]:
# Cell 2: Initialize Kùzu database
import kuzu
import os
import shutil
import pandas as pd

# Step 1: Initialize database
db_path = os.path.join("..", "db/graph_db")
shutil.rmtree(db_path, ignore_errors=True)
db = kuzu.Database(db_path)
conn = kuzu.Connection(db)
print(f"Database initialized at {db_path}")

# Step 2: Define schema
conn.execute("""
    CREATE NODE TABLE Account(
        id STRING,
        country STRING,
        total_inflow DOUBLE DEFAULT 0.0,
        total_outflow DOUBLE DEFAULT 0.0,
        PRIMARY KEY(id)
    )
""")
conn.execute("""
    CREATE REL TABLE Transaction(
        FROM Account TO Account,
        transaction_id STRING,
        amount DOUBLE,
        timestamp STRING,
        transaction_type STRING,
        sender_country STRING,
        receiver_country STRING,
        description STRING,
        suspicious_flag BOOLEAN
    )
""")
print("Schema created: Account and Transaction tables")


RuntimeError: IO exception: Could not set lock on file : c:\Users\cha\Desktop\aml_solution\db\graph_db
See the docs: https://docs.kuzudb.com/concurrency for more information.

In [22]:
# Step 3: Load data
csv_path = "synthetic_transactions.csv"
data = pd.read_csv(csv_path)
print(f"Loaded CSV with {len(data)} transactions")

# Extract unique accounts
accounts = pd.concat([data["Sender_ID"], data["Receiver_ID"]]).unique()
sender_country_map = data[["Sender_ID", "Sender_Country"]].drop_duplicates().set_index("Sender_ID")["Sender_Country"]
account_df = pd.DataFrame({
    "id": accounts,
    "country": [sender_country_map.get(acc, "Unknown") for acc in accounts],
    "total_inflow": 0.0,
    "total_outflow": 0.0
})
print(f"Extracted {len(account_df)} unique accounts")
conn.execute("COPY Account FROM account_df")

# Prepare and load transactions
transaction_df = data[[
    "Transaction_ID", "Sender_ID", "Receiver_ID", "Amount", "Timestamp",
    "Transaction_Type", "Sender_Country", "Receiver_Country", "Description", "Suspicious_Flag"
]].rename(columns={
    "Transaction_ID": "transaction_id",
    "Sender_ID": "from_id",
    "Receiver_ID": "to_id",
    "Amount": "amount",
    "Timestamp": "timestamp",
    "Transaction_Type": "transaction_type",
    "Sender_Country": "sender_country",
    "Receiver_Country": "receiver_country",
    "Description": "description",
    "Suspicious_Flag": "suspicious_flag"
})
temp_csv = "temp_transactions.csv"
transaction_df.to_csv(temp_csv, index=False)
conn.execute(f"""
    COPY Transaction FROM '{temp_csv}' (HEADER=true)
""")
os.remove(temp_csv)
print("Data loaded successfully")

Loaded CSV with 12686 transactions
Extracted 1000 unique accounts


RuntimeError: Copy exception: Found duplicated primary key value E0314, which violates the uniqueness constraint of the primary key column.

In [23]:

# Step 4: Update computed properties
conn.execute("MATCH (a:Account)-[t:Transaction]->() SET a.total_outflow = a.total_outflow + t.amount")
conn.execute("MATCH ()-[t:Transaction]->(a:Account) SET a.total_inflow = a.total_inflow + t.amount")
print("Computed properties updated")


Computed properties updated


In [24]:

# Step 5: Verify data
result = conn.execute("MATCH (a:Account) RETURN COUNT(a) AS account_count")
print(f"Total accounts: {result.get_next()[0]}")
result = conn.execute("MATCH ()-[t:Transaction]->() RETURN COUNT(t) AS transaction_count")
print(f"Total transactions: {result.get_next()[0]}")
result = conn.execute("""
    MATCH (a:Account)-[t:Transaction {suspicious_flag: true}]->(b:Account)
    RETURN t.transaction_id, a.id AS sender, b.id AS receiver, t.amount, t.timestamp
    LIMIT 5
""")
print("Sample suspicious transactions:")
print(result.get_as_df())
result = conn.execute("""
    MATCH (a:Account)-[t:Transaction]->()
    RETURN a.id, COUNT(t) AS out_degree
    ORDER BY out_degree DESC
    LIMIT 5
""")
print("Top 5 accounts by outgoing transactions:")
print(result.get_as_df())

Total accounts: 1000
Total transactions: 0
Sample suspicious transactions:
Empty DataFrame
Columns: [t.transaction_id, sender, receiver, t.amount, t.timestamp]
Index: []
Top 5 accounts by outgoing transactions:
Empty DataFrame
Columns: [a.id, out_degree]
Index: []
